In [44]:
import yaml
import pandas as pd

# dataset 불러오기
with open("config/default.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)
    train_paths = config["data"]["train_path"]
    supplement_paths = config['data']['supplement_path']

dfs = [pd.read_csv(p) for p in supplement_paths]
df = pd.read_csv(train_paths)

# 불러온 데이터프레임 확인 (선택 사항)
# for i, df in enumerate(dfs):
#    print(f"DataFrame {i+1} loaded with {len(df)} rows.")

# 만약 하나의 데이터프레임으로 합치고 싶다면
combined_df = pd.concat(dfs, ignore_index=True)
combined_df = pd.concat([df, combined_df], ignore_index=True)
print("\nCombined DataFrame head:")
print(combined_df.info())


Combined DataFrame head:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16963 entries, 0 to 16962
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       7973 non-null   float64
 1   SMILES   16963 non-null  object 
 2   Tg       557 non-null    float64
 3   FFV      7892 non-null   float64
 4   Tc       1611 non-null   float64
 5   Density  613 non-null    float64
 6   Rg       614 non-null    float64
dtypes: float64(6), object(1)
memory usage: 927.8+ KB
None


In [46]:
from rdkit import Chem

def canonicalize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return Chem.MolToSmiles(mol, canonical=True)
        else:
            return None
    except:
        return None

combined_df['canonical_smiles'] = combined_df['SMILES'].apply(canonicalize_smiles)
combined_df.dropna(subset=['canonical_smiles'], inplace=True)

# 3. 데이터가 많은 행을 우선순위로 두기 위한 정렬 (이전과 동일)
combined_df['non_na_count'] = combined_df.notna().sum(axis=1)
# non_na_count가 높은 순으로 정렬. 이래야 'first'를 쓸 때 데이터가 많은 행이 선택됨
combined_df.sort_values('non_na_count', ascending=False, inplace=True)


# 4. 그룹별 집계(Aggregation) 로직 정의 및 실행
#   - Tc: 평균(mean)을 계산
#   - 나머지 모든 열: 첫 번째 값(first)을 선택 (3번에서 정렬했으므로 데이터가 가장 많은 행의 값이 됨)

# 집계 규칙을 담을 딕셔너리 생성
agg_rules = {}
for col in combined_df.columns:
    if col not in ['canonical_smiles', 'non_na_count']: # 작업용 열은 제외
        if col == 'Tc':
            agg_rules[col] = 'mean' # Tc는 평균값으로
        else:
            agg_rules[col] = 'first' # 나머지는 첫번째 값으로

# groupby와 agg를 이용해 새로운 데이터프레임 생성
df_agg = combined_df.groupby('canonical_smiles').agg(agg_rules).reset_index()


# 5. 최종 결과 확인
# 컬럼 순서를 원본과 유사하게 재정렬
final_cols = ['SMILES'] + [col for col in combined_df.columns if col not in ['SMILES', 'canonical_smiles', 'non_na_count']]
df_final = df_agg.rename(columns={'canonical_smiles': 'SMILES_canonical'})[final_cols]


print("===== 중복 처리 후 최종 데이터프레임 =====")
print(df_final.info())

===== 중복 처리 후 최종 데이터프레임 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10343 entries, 0 to 10342
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SMILES   10343 non-null  object 
 1   id       7973 non-null   float64
 2   Tg       557 non-null    float64
 3   FFV      7892 non-null   float64
 4   Tc       866 non-null    float64
 5   Density  613 non-null    float64
 6   Rg       614 non-null    float64
dtypes: float64(6), object(1)
memory usage: 565.8+ KB
None


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 분석할 물성치 컬럼 리스트
properties = ["Tg", "FFV", "Tc", "Density", "Rg"]

# seaborn 스타일 적용으로 더 나은 시각화 품질 확보
sns.set_style("whitegrid")

# 각 속성에 대해 히스토그램 그리기
for prop in properties:
    plt.figure(figsize=(8, 5)) # 플롯 사이즈 지정

    # 해당 컬럼에서 결측치(NaN)를 제외하고 데이터 선택
    data_to_plot = df_final[prop].dropna()

    # 히스토그램 생성
    sns.histplot(data_to_plot, kde=True, bins=30) # kde=True로 밀도 플롯 추가

    # 그래프 제목 및 라벨 설정
    plt.title(f'Histogram of {prop}', fontsize=15)
    plt.xlabel(prop, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)

    # 그래프 출력
    plt.show()

In [ ]:
# 각 속성에 대해 상자 그림 그리기
for prop in properties:
    plt.figure(figsize=(6, 6)) # 플롯 사이즈 지정

    # 상자 그림 생성
    sns.boxplot(y=df[prop])

    # 그래프 제목 및 라벨 설정
    plt.title(f'Box Plot of {prop}', fontsize=15)
    plt.ylabel(prop, fontsize=12)

    # 그래프 출력
    plt.show()